# Numeric data

In this notebook we talk about some of the functions and methods that are commonly used to deal with numeric columns in our data, in preparation to doing machine learning. Numeric columns can be **integer**, **float**, or **datetime** columns.

As usual, let's begin by reading some data.

In [1]:
import pandas as pd

auto = pd.read_csv('data/auto-mpg.csv', sep = '\s+', header = None, 
                   names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 
                            'acceleration', 'model year', 'origin', 'car_name'])
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


Let's check the data types:

In [ ]:
auto.dtypes

We can see that we have quite a few numeric columns in here. However, we also need to be careful: a column like `cylinders` looks numeric, but probably should be treated as a categorical column. Why? As yourself which one of these two is more likely:

- do any kind of math with `cylinders`, such as taking the log of it, adding it to another column, or being interested in knowing the average number of cylinders?
- use cylinder as a grouping variable, such as wanting to see the average horsepower **by cylinder**?

We can probably agree that the second case is more likely. For this reason, we need convert cylinder into a categorical column. More on categorical columns in the next lesson.

In [ ]:
auto['cylinders'] = auto['cylinders'].astype('category')

Let's now look at some distributions for our numeric columns.

In [ ]:
import seaborn as sns

sns.pairplot(auto);

### Exercise

We can see that `weight` and `displacement` have a **positive relationship** (as one goes up, so does the other, and vice versa). We may wonder if `cylinders` plays a part.

Import the `seaborn` library and plot the scatter plot between `weight` and `displacement`, then use the `hue` argument to color-code it by `cylinders`.

### End of exercise

Let's look at some summary statistics for `acceleration`. One quick and easy way to do that is using `describe`.

In [ ]:
results_all = auto.describe()
results_all

Notice that the result of `describe` itself looks look a `DataFrame`. That means that if we wanted to extract certain pieces out of it, we should be able to do it using the methods we learned for working with `DataFrame` objects.

### Exercise

Extract the mean and standard deviation for acceleration from the results above and store them in variables called `acc_mean` and `acc_std`.

### End of exercise

There are other functions we can use to get additional summary statistics from the data. For example, notice that `describe` only shows us the 25th, 50th, and 75th **percentiles**. The $p$th percentile is **a value such that $p$ percent of the data is below that value**, which means the remaining $1-p$ percent of the data is above that value. For example, if the 25th percentile for acceleration `mpg` is 17.5, then 25 percent of cars in our data have `mpg` below 17.0. The 50th percentile is also known as the **median**.

Looking at percentiles helps us get a feel for the distribution of the data. For example, looking at very low or very high percentiles can help us identify **outliers** or **extreme values**. We can use the `quantile` method to get any percentile we want. Here's how:

In [ ]:
auto.quantile(q = [0.05, 0.25, 0.50, 0.75, 0.95])

Now let's see how we can obtain these same summary statistics, but grouped by `cylinders`. There is already one way we know how to do that: we can first subset the data by `cylinders == 3` and run the summary statistic, then do the same for `cylinders == 4`, etc.

In [ ]:
auto.loc[auto['cylinders'] == 3, :].describe()

But of course the above way is tedious. Instead, we can use the `groupby` method to do it all at once. For example, here's the average of each column grouped by `cylinders`.

In [ ]:
auto.groupby('cylinders').mean()

A slightly confusing thing happens if we use `describe` instead of `mean`:

In [ ]:
auto.groupby('cylinders').describe()

The problem with the above table is that there's too much information packed in it. Each value of `cylinders` has its own row, which is fine. But the columns look like they have a hierarchical structure, which makes sense because we asked for many summary statistics for each column.

So what if for example we wanted to store all the results from the above table, and then look at the results for `acceleration` only. Turns out we can do the same thing we usually do:

In [ ]:
results_all = auto.groupby('cylinders').describe()
results_all['acceleration']

The difference is that `results_all['acceleration']` isn't just a single column, but many columns. That's because `results_all` has hierarchical columns. So if we want to drill further down to a specific column, such as `std`, we need to go one level deeper.

In [ ]:
results_all['acceleration']['std']

So far it looks like hierarchical columns are not that complicated. However, things do get a little confusing when we want to cross the hierarchy the other way around. For example, let's say we want to quickly compare the standard deviations of all our columns, grouped by `cylinders`. This information is in the above table, but how do we extract it?

It turns out that to do this, we need to use the `IndexSlice` function in `pandas`. Here how it works:
- we use `loc` to say which rows and columns we want
- we use `slice(None)` to say we want everything at a given hierarchy
- if we want to filter at any level of hierarchy, we provide the index we want to filter by

In [ ]:
from pandas import IndexSlice as idx
results_all.loc[slice(None) , idx[slice(None), 'std']]

By the way, the same way that we can have hierarchical columns, we can also have hierarchical rows, more specifically hierachical row indexes. Hierarchical indexes are called `MultiIndex` in `pandas`. This is a more advanced topic and we will leave it at that for now.

### Exercise

We saw how a histogram shows us counts for **evenly-sized** intervals of a numeric column. What if we wanted to see counts for any intervals we specify, evenly-sized or not? Here's an example:

Find out how many cars have `mpg` less than 18, between 18 and 25, and 25 or more?

Note that there are many ways of doing this, so to narrow it down a little do this:
- use `loc` to limit data to the above ranges
- use the `shape` method to get row counts

Check what happens if some of your data points fall outside of any of intervals. Does this match what you would expect?

### End of exercise

There is another, much faster way to solve the exercise above: by using the `pd.cut` function and specify the cut-offs using the `bins` argument.

In [ ]:
auto['mpg_cat'] = pd.cut(auto['mpg'], bins = [0, 18, 25, 100])
auto[['mpg', 'mpg_cat']].head()

Now we can just use `value_counts` to get our counts.

In [ ]:
auto['mpg_cat'].value_counts()

### Exercise

Let's create `mpg_cat` again, but make the following changes to it: 

- Note that the choice of using 0 and 100 as the lower and upper bounds is a little arbitrary. So use `-np.Inf` and `np.Inf` instead.
- By default, cut labels the catgories based on the interval range it covers, but we can change the labels to anything we like using the `labels` argument. Rename the labels to `low`, `med` and `high`.

### End of exercise

The `pd.cut` function is different from the other functions (or methods) we learned about so far. The `describe`, `mean`, `quantile`, or `value_counts` functions are **feature summarization** functions, but `pd.cut` is a **feature transformation** function, meaning that not only its input is a feature, but its output is also a feature. In the above example, the input to `pd.cut` was `mpg`, and the output was the `mpg_cat` which we appended to the data as a new column.

There are many other **feature transformation** functions. In fact, **feature engineering** consists of running these feature transformations on the data and gradually modifying existing columns and adding new features to the data, with the goal of ending up with features that are more useful to the model than the original features we started with.

Another common type of feature transformation for numeric features is **feature normalization**. Note that **normalization** is a word that means something very different in relational databases than in machine learning, so be careful not to confuse the two. The two most common ways to normalize features is **Z-normalization** and **min-max normalization**:

- **Z-normalization** consists of the following transformation, and results in most of the values for the transformed $x$ being between -2 and 2. 
$$x \rightarrow \dfrac{x - mean(x)}{std(x)}$$
- **min-max normalization** consists of the following transformation, and forces all the values of the transformed $x$ to be between 0 and 1: 
$$x \rightarrow \dfrac{x - min(x)}{max(x) - min(x)}$$

So what is the purpose of normalization? If we only have one feature, normalization doesn't really serve a purpose. Normalization makes sense when we have many features and we want to **put them on the same scale**, which is why normalization is also sometimes called **rescaling** or **standardization**. 

**Some (but not all) ML algorithms only work properly if the data is normalized, otherwise the features that are on a larger scale will exert undue influence on the model.** So normalization acts as a way of leveling the playing field.

### Exercise

Normalize `mpg`, `displacement`, `weight` and `acceleration`. Instead of overwriting the original columns, add the normalized features as new columns and name each column using the column name and a `_norm` suffix. You are free to choose between Z-normalization or min-max normalization.

Unless your Python skills are improving by leaps and bounds, you probably normalized the features one at a time. What if we wanted to do it all at once? Write a loop to iterate over the four columns and normalize each. To make it easier, we already put the column names in a list for you.

In [ ]:
num_cols = ['mpg', 'displacement', 'weight', 'acceleration']

There is an even better way to run our transformations all at once without writing a loop. First we have to write a function whose input is an array and whose output is an array of the same size with the values normalized. Write such a function and use the below cell to test it and make sure it worked.

In [ ]:
def normalize(x):
    x_norm = x # write your function
    return x_norm

x_test = np.array([3, 5, 9, 11, 2, 0])
normalize(x_test)

Time to apply the function to the data. To apply the function, we use the `apply` method.
- note that we need to limit the data to only the four columns we wish to transform
- we need to use the `axis = 1` argument to let `apply` know that the transformation applies to columns (`axis = 0` would apply it to rows, which is not what we want here)

Apply the transformation to the data and check the results using `head`. Then do an additional sanity check:
- if your function is doing Z-normalization, then check the mean and standard deviation of your normalized columns to make sure they are 0 and 1 respectively
- if your function is doing min-max normalization, then check the minimum and maximum values of your normalized columns to make sure they are 0 and 1 respectively

### End of exercise

Since normalization is a very common task, you shoudn't be surprised to find out that there is already a function for doing this. I hope you still found the previous exercise useful!

However, this time we have to go to the `sklearn` library to find our function. The `sklearn` library is Python's most common machine learning library and one that we will return to in future lectures. In addition to the machine learning algorithms we will learn about, `sklearn` also has functions for **pre-processing data**, which is a vague term that includes tasks such as missing-value imputation, feature engineering and so on. Let's see how we can use it to normalize our data.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
num_cols_norm = [c + '_norm' for c in num_cols] # names of transformed columns

At first blush, this code might look a little strange, but this pattern as we will see is very common to ML-related tasks in `sklearn`:
- initialize the process by choosing the function (with arguments we wish, if any)
- run `fit` first on the data, followed by `transform` to apply the transformation

In [ ]:
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(auto[num_cols])
auto[num_cols_norm] = pd.DataFrame(minmax_scaler.transform(auto[num_cols]))
auto[num_cols_norm].head()

In [ ]:
sns.pairplot(auto[num_cols_norm]);

Here's the same example, but using Z-normalization.

In [ ]:
znorm_scaler = StandardScaler()
znorm_scaler.fit(auto[num_cols])
auto[num_cols_norm] = pd.DataFrame(znorm_scaler.transform(auto[num_cols]))
auto[num_cols_norm].head()

Let's look at the scatter plot matirx for the normalized features.

In [ ]:
sns.pairplot(auto[num_cols_norm]);

In either case, it doesn't look like normalization did anything to make our scatter plot matrix looks different. Do you notice what changed? The answer is that the **scale of the data** is what changed. Just check the $x$ and $y$ axes and you'll see. Normalization is not really supposed to change the nature of the data, just put all features on the same scale.

Let's look one last time at the code for normalizing the data using `sklearn`.

In [ ]:
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(auto[num_cols])
auto[num_cols_norm] = pd.DataFrame(minmax_scaler.transform(auto[num_cols]))
auto[num_cols_norm].head()

There's two things we need to point out about the code above:

In the third line, notice how we used `pd.DataFrame` to turn the data into a `DataFrame` object. This is because `sklearn` returns a `numpy` array instead of a `DataFrame`. As we saw earlier, the distinction is most of the time unimportant since a `DataFrame` is just a sort of wrapper for a `numpy` array, but if we want to use `DataFrame` methods like `head` to examine the results, we need to do our own conversion.

You might be curious why we use `fit` followed by `transform`. What exactly happens when we run `fit`? Why should those two not be steps not be a single step? The answer to that will become obvious later in the context of machine learning, but here's a short answer using `MinMaxScaler` as our example:
  - When we run `fit` we find the min and max for the columns and rememeber it.
  - When we run `transform` we apply the transformation using the min and max we found when we ran `fit`.

This means that we can learn the min and max once, and then apply the **same** transformation (with the same min and max) not just to the original data, but any future data. In machine learning, this has important consequences, but that's the topic of a future lecture.